1. data [x]
    - get the vrts for esa,ldar,s1,edem
    - get the tifs from tls and rng 

2. datapre [+]
    - clip by bbox and 
    - split train from test (cpu modelling continual ml?)

3. datamod [tab]
    - step 1: predit ldem 
    - step 2: get scores for with and without residual 
    - step 3: style transfer (+residual)
    - step 4: expand prediction range by N pixels same model 
    - step 5: data fusion with ldem
4. datamod [img]

In [1]:
from glob import glob
import os

In [2]:
outdir4749 =  "/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/reproj4749"  
ldem_vrt = f"{outdir4749}/ldem_4749.vrt"
cmd = f"gdalbuildvrt -overwrite {ldem_vrt} {outdir4749}/*.tif"
os.system(cmd)

0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [3]:
esawc_vrt = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRT/esawc/esawc.vrt"
s1_vrt = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRT/s1/s1.vrt"
edem_egm_vrt = "/media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRT/edem_egm/edem_egm.vrt" 

tls_fn = "/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/reproj4749/AngkorDTM50cm.tif"
rng_files = glob("/media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/reproj4749/*.tif")
rgn_files = [f for f in rng_files if "AngkorDTM50cm" not in f and "delta_s_m2070" not in f]

vrt_files = [esawc_vrt, s1_vrt,edem_egm_vrt, ldem_vrt] # these are the vrt files 
temp_dir = 'temp'
os.makedirs(temp_dir, exist_ok=True)

esawc_vrt, s1_vrt,edem_egm_vrt  and tls_fn #set for self supervision SSL 

In [4]:
import rasterio
from rasterio.windows import from_bounds
import os
import tempfile
import shutil

### use also xres and yres from tls_fn in addition to bbox so that the out are aligned with same spatial info and pixel counts
def process_raster_data(tls_fn, vrt_files, temp_dir):
    """
    Reads data from VRT files, aligning their spatial information and pixel counts
    with a GeoTIFF file, and saves the extracted data as new GeoTIFF files.

    Args:
        tls_fn (str): Path to the reference GeoTIFF file.
        vrt_files (list): List of paths to the VRT files to process.
        temp_dir (str): Path to the directory where the output GeoTIFF files will be saved.
    """
    # Ensure the output directory exists
    os.makedirs(temp_dir, exist_ok=True)

    try:
        # Open the reference GeoTIFF file to get spatial information
        with rasterio.open(tls_fn) as tls_dataset:
            # Get the bounding box, transform, width, and height of the reference GeoTIFF
            bbox = tls_dataset.bounds
            transform = tls_dataset.transform
            out_width = tls_dataset.width
            out_height = tls_dataset.height
            print(f"Reference GeoTIFF: {tls_fn}")
            print(f"  Bounding box: {bbox}")
            print(f"  Transform: {transform}")
            print(f"  Width: {out_width}, Height: {out_height}")

            # Process each VRT file
            for vrt_file in vrt_files:
                try:
                    # Open the VRT file
                    with rasterio.open(vrt_file) as vrt_dataset:
                        print(f"\nProcessing {vrt_file}...")

                        # Calculate the window based on the bounding box and transform of the reference GeoTIFF
                        window = from_bounds(bbox.left, bbox.bottom, bbox.right, bbox.top, vrt_dataset.transform)

                        # Read the data within the calculated window
                        # Specify the desired output shape to ensure alignment
                        try:
                            data = vrt_dataset.read(window=window, out_shape=(vrt_dataset.count, out_height, out_width))
                        except ValueError as e:
                            print(f"  Error reading data from {vrt_file} within the window: {e}")
                            print("  This might indicate a significant spatial mismatch.")
                            data = None

                        if data is not None:
                            # Update the metadata for the output GeoTIFF file
                            out_meta = vrt_dataset.meta.copy()
                            out_meta.update({
                                "driver": "GTiff",
                                "height": out_height,
                                "width": out_width,
                                "transform": transform,
                                "dtype": data.dtype,
                            })

                            # Check for valid dimensions before writing
                            if out_height > 0 and out_width > 0:
                                # Create the output file name
                                output_filename = os.path.join(temp_dir, f"aligned_{os.path.basename(vrt_file)}")
                                output_filename = os.path.splitext(output_filename)[0] + ".tif"

                                # Write the data to a new GeoTIFF file
                                with rasterio.open(output_filename, "w", **out_meta) as dst:
                                    dst.write(data)
                                print(f"  Saved aligned data to {output_filename}")
                            else:
                                print(f"  Skipping {vrt_file}: Output dimensions are invalid (height={out_height}, width={out_width}).")
                                output_filename = os.path.join(temp_dir, f"aligned_{os.path.basename(vrt_file)}")
                                output_filename = os.path.splitext(output_filename)[0] + ".tif"
                                shutil.copyfile(tls_fn, output_filename)
                                print(f"  Copied {tls_fn} to {output_filename} as a fallback.")
                        else:
                            print(f"  Skipping {vrt_file} due to issues reading data.")
                            output_filename = os.path.join(temp_dir, f"aligned_{os.path.basename(vrt_file)}")
                            output_filename = os.path.splitext(output_filename)[0] + ".tif"
                            shutil.copyfile(tls_fn, output_filename)
                            print(f"  Copied {tls_fn} to {output_filename} as a fallback.")

                except rasterio.RasterioIOError as e:
                    print(f"  Error processing {vrt_file}: {e}")
                    print(f"  Skipping {vrt_file} and continuing. Check if the file exists and is a valid raster.")
                except Exception as e:
                    print(f"  An unexpected error occurred while processing {vrt_file}: {e}")

    except rasterio.RasterioIOError as e:
        print(f"Error opening {tls_fn}: {e}")
        print("Please ensure the file path is correct and the file is a valid GeoTIFF.")
    except Exception as e:
        print(f"An unexpected error occurred with the reference GeoTIFF: {e}")



if __name__ == '__main__':
  
    # Create a temporary directory if it doesn't exist
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    process_raster_data(tls_fn, vrt_files, temp_dir)

    # Clean up the temporary directory after processing (optional)
    # shutil.rmtree(temp_dir)

Reference GeoTIFF: /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/reproj4749/AngkorDTM50cm.tif
  Bounding box: BoundingBox(left=103.73573115613353, bottom=13.317802380912203, right=103.99762004502242, top=13.47535793646776)
  Transform: | 0.00, 0.00, 103.74|
| 0.00,-0.00, 13.48|
| 0.00, 0.00, 1.00|
  Width: 2357, Height: 1418

Processing /media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRT/esawc/esawc.vrt...
  Saved aligned data to temp/aligned_esawc.tif

Processing /media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRT/s1/s1.vrt...
  Saved aligned data to temp/aligned_s1.tif

Processing /media/ljp238/12TBWolf/ARCHIEVE/ARCHIVE_VRT/edem_egm/edem_egm.vrt...
  Saved aligned data to temp/aligned_edem_egm.tif

Processing /media/ljp238/12TBWolf/ARCHIEVE/LIDAR_DTM/reproj4749/ldem_4749.vrt...
  Saved aligned data to temp/aligned_ldem_4749.tif


In [7]:
import rasterio
import os
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import joblib
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


def load_geotiff(file_path):
    dataset = rasterio.open(file_path)
    data = dataset.read()
    return dataset, data


def preprocess_data(s1_data, esawc_data, ldem_data):
    s1_flat = s1_data.reshape(s1_data.shape[0], -1).T
    esawc_flat = esawc_data.reshape(esawc_data.shape[0], -1).T
    ldem_flat = ldem_data.reshape(ldem_data.shape[0], -1).T

    if s1_flat.shape[0] != esawc_flat.shape[0] or s1_flat.shape[0] != ldem_flat.shape[0]:
        raise ValueError("Number of pixels in s1, esawc, and ldem do not match.")

    features = np.concatenate((s1_flat, esawc_flat), axis=1)
    valid_indices = ~np.isnan(ldem_flat).any(axis=1)
    features_filtered = features[valid_indices]
    labels_filtered = ldem_flat[valid_indices].ravel()
    return features_filtered, labels_filtered



def train_model(features, labels):
    X_train, X_test, y_train, y_test = train_test_split(
        features, labels, test_size=0.2, random_state=42
    )

    model = lgb.LGBMRegressor(random_state=42)
    model.fit(features, labels)
    score = model.score(X_test, y_test)
    logger.info(f"Model R^2 Score: {score}")
    return model



def predict_and_save(model, features, edem_dataset, output_path):
    edem_data = edem_dataset.read()
    predictions_flat = model.predict(features)
    predictions = predictions_flat.reshape(edem_data.shape[1], edem_data.shape[2])
    profile = edem_dataset.profile.copy()
    profile.update(
        dtype=rasterio.float32,
        count=1,
        height=predictions.shape[0],
        width=predictions.shape[1],
    )
    with rasterio.open(output_path, "w", **profile) as dst:
        dst.write(predictions, 1)


# add timer when bfore and after the function 
# add other function to select the model if RF, XGBoost, CatBoost, LightGBM, etc
# add option to chose features , e.g. [s1, esawc, edem] or [s1,esawc], or ['edem'] and only select the features give 
# save the model with paramers as name of the features and num_of_iterations 
def train_and_predict_lightgbm(temp_dir, output_dir, output_filename="lgbm_predictions.tif"):
    os.makedirs(output_dir, exist_ok=True)

    s1_path = os.path.join(temp_dir, "aligned_s1.tif")
    esawc_path = os.path.join(temp_dir, "aligned_esawc.tif")
    ldem_path = os.path.join(temp_dir, "aligned_ldem_4749.tif")
    edem_path = os.path.join(temp_dir, "aligned_edem_egm.tif")

    try:
        s1_dataset, s1_data = load_geotiff(s1_path)
        esawc_dataset, esawc_data = load_geotiff(esawc_path)
        ldem_dataset, ldem_data = load_geotiff(ldem_path)
        edem_dataset, edem_data = load_geotiff(edem_path)

        features, labels = preprocess_data(s1_data, esawc_data, ldem_data)

        model = train_model(features, labels)

        model_filename = os.path.join(output_dir, "lightgbm_model.joblib")
        joblib.dump(model, model_filename)

        output_path = os.path.join(output_dir, output_filename)
        predict_and_save(model, features, edem_dataset, output_path)

    except Exception as e:
        logger.error(f"An error occurred: {e}")
    finally:
        if 's1_dataset' in locals(): s1_dataset.close()
        if 'esawc_dataset' in locals(): esawc_dataset.close()
        if 'ldem_dataset' in locals(): ldem_dataset.close()
        if 'edem_dataset' in locals(): edem_dataset.close()



# if __name__ == '__main__':
#     output_dir = os.path.join(temp_dir, "output")
#     train_and_predict_lightgbm(temp_dir, output_dir)


In [ ]:
# go through the temp directory load s1 and esacf as features and ldem as label
# remove the missing values filtering out the no data values by ldem 
# fit random forest model 
# save the model 
# make predictions on the entire feature set 
# write the predictions to a new tiff file

In [ ]:
import os
import time
import logging
import rasterio
import numpy as np
import joblib

import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Setup logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def timer(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        logger.info(f"Starting {func.__name__}...")
        result = func(*args, **kwargs)
        end = time.time()
        logger.info(f"Finished {func.__name__} in {end - start:.2f} seconds.")
        return result
    return wrapper

@timer
def load_geotiff(file_path):
    dataset = rasterio.open(file_path)
    data = dataset.read()
    return dataset, data

def select_features(feature_list, datasets):
    feature_arrays = []
    for feature in feature_list:
        data = datasets[feature]
        feature_arrays.append(data.reshape(data.shape[0], -1).T)
    combined_features = np.concatenate(feature_arrays, axis=1)
    return combined_features

@timer
def preprocess_data(feature_list, datasets, ldem_data):
    features = select_features(feature_list, datasets)
    ldem_flat = ldem_data.reshape(ldem_data.shape[0], -1).T
    if features.shape[0] != ldem_flat.shape[0]:
        raise ValueError("Mismatch in number of pixels between features and labels.")
    valid_indices = ~np.isnan(ldem_flat).any(axis=1)
    features_filtered = features[valid_indices]
    labels_filtered = ldem_flat[valid_indices].ravel()
    return features_filtered, labels_filtered

def get_model(model_type):
    if model_type == "lightgbm":
        return lgb.LGBMRegressor(random_state=42)
    elif model_type == "xgboost":
        return xgb.XGBRegressor(random_state=42)
    elif model_type == "catboost":
        return cb.CatBoostRegressor(verbose=0, random_state=42)
    elif model_type == "rf":
        return RandomForestRegressor(random_state=42)
    else:
        raise ValueError(f"Unsupported model type: {model_type}")

@timer
def train_model(features, labels, model_type="lightgbm"):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    model = get_model(model_type)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    logger.info(f"{model_type.upper()} Model R^2 Score: {score:.4f}")
    return model

@timer
def predict_and_save(model, features, edem_dataset, output_path):
    edem_data = edem_dataset.read()
    predictions_flat = model.predict(features)
    predictions = predictions_flat.reshape(edem_data.shape[1], edem_data.shape[2])
    
    profile = edem_dataset.profile.copy()
    profile.update(dtype=rasterio.float32, count=1, height=predictions.shape[0], width=predictions.shape[1])
    
    with rasterio.open(output_path, "w", **profile) as dst:
        dst.write(predictions, 1)

@timer
def train_and_predict(temp_dir, output_dir, 
                      output_filename="predictions.tif", 
                      feature_list=["s1", "esawc"], 
                      model_type="lightgbm"):
    
    os.makedirs(output_dir, exist_ok=True)

    file_paths = {
        "s1": os.path.join(temp_dir, "aligned_s1.tif"),
        "esawc": os.path.join(temp_dir, "aligned_esawc.tif"),
        "ldem": os.path.join(temp_dir, "aligned_ldem_4749.tif"),
        "edem": os.path.join(temp_dir, "aligned_edem_egm.tif")
    }

    datasets = {}
    try:
        # Load datasets
        for key in feature_list + ["ldem", "edem"]:
            datasets[key], datasets[key + "_data"] = load_geotiff(file_paths[key])

        # Preprocess data
        features, labels = preprocess_data(feature_list, {k: datasets[k + "_data"] for k in feature_list}, datasets["ldem_data"])

        # Train model
        model = train_model(features, labels, model_type=model_type)

        # Save model
        feature_str = "_".join(feature_list)
        model_filename = f"{model_type}_{feature_str}_{len(labels)}samples.joblib"
        model_filepath = os.path.join(output_dir, model_filename)
        joblib.dump(model, model_filepath)
        logger.info(f"Model saved to {model_filepath}")

        # Predict and save output
        output_path = os.path.join(output_dir, output_filename)
        predict_and_save(model, features, datasets["edem"], output_path)

    except Exception as e:
        logger.error(f"An error occurred: {e}")
    finally:
        # Close datasets
        for key in datasets:
            if isinstance(datasets[key], rasterio.io.DatasetReader):
                datasets[key].close()

if __name__ == '__main__':
    output_dir = os.path.join(temp_dir, "output")
    train_and_predict(
        temp_dir, 
        output_dir, 
        output_filename="lightgbm_edem_predictions.tif",
        feature_list=["edem"],
        model_type="lightgbm"   # Options: "lightgbm", "xgboost", "catboost", "rf"
    )

# add resisuals 
#modely type and feature list should be on top and a function that attributtes the model_finaname

2025-04-28 18:39:48,874 - INFO - Starting train_and_predict...
2025-04-28 18:39:48,876 - INFO - Starting load_geotiff...
2025-04-28 18:39:48,901 - INFO - Finished load_geotiff in 0.03 seconds.
2025-04-28 18:39:48,902 - INFO - Starting load_geotiff...
2025-04-28 18:39:48,926 - INFO - Finished load_geotiff in 0.02 seconds.
2025-04-28 18:39:48,927 - INFO - Starting load_geotiff...


2025-04-28 18:39:48,956 - INFO - Finished load_geotiff in 0.03 seconds.
2025-04-28 18:39:48,958 - INFO - Starting preprocess_data...
2025-04-28 18:39:49,022 - INFO - Finished preprocess_data in 0.06 seconds.
2025-04-28 18:39:49,023 - INFO - Starting train_model...


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007877 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 2673780, number of used features: 1
[LightGBM] [Info] Start training from score 58726557525220922143210296142894989312.000000


2025-04-28 18:39:51,104 - INFO - LIGHTGBM Model R^2 Score: -0.6119
2025-04-28 18:39:51,106 - INFO - Finished train_model in 2.08 seconds.
2025-04-28 18:39:51,111 - INFO - Model saved to temp/output/lightgbm_edem_3342226samples.joblib
2025-04-28 18:39:51,111 - INFO - Starting predict_and_save...
2025-04-28 18:39:52,244 - INFO - Finished predict_and_save in 1.13 seconds.
2025-04-28 18:39:52,247 - INFO - Finished train_and_predict in 3.37 seconds.


In [26]:
#w943 and h567 :: create DL datatraining with numpy equal size patches 

In [28]:
# 4749